In [2]:
import numpy
import pandas
from keras.models import Sequential
from keras import models
from keras import layers
from keras.layers import Dense
from keras.wrappers.scikit_learn import KerasClassifier
from keras.utils import np_utils
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
from sklearn.pipeline import Pipeline


C:\Users\dell\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [3]:

# fix random seed for reproducibility
seed = 7
numpy.random.seed(seed)


In [4]:
# load dataset
dataframe = pandas.read_csv("iris.csv", header=None)
from sklearn.utils import shuffle
dataframe = shuffle(dataframe)
dataset = dataframe.values
X = dataset[:,0:4].astype(float)
Y = dataset[:,4]


In [5]:


# encode class values as integers
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)


In [26]:
encoded_Y

array([2, 1, 0, 1, 2, 0, 1, 1, 0, 1, 1, 1, 0, 2, 0, 1, 2, 2, 0, 0, 1, 2,
       1, 2, 2, 2, 1, 1, 2, 2, 2, 1, 0, 2, 1, 0, 0, 0, 0, 2, 2, 1, 2, 2,
       1, 0, 1, 1, 2, 0, 0, 0, 2, 0, 2, 1, 1, 1, 0, 0, 0, 1, 2, 1, 1, 0,
       2, 0, 0, 2, 2, 0, 2, 0, 1, 2, 1, 0, 1, 0, 2, 2, 1, 0, 0, 1, 2, 0,
       2, 2, 1, 0, 1, 0, 2, 2, 0, 0, 2, 1, 2, 2, 1, 0, 0, 2, 0, 0, 1, 2,
       2, 1, 1, 0, 2, 0, 0, 1, 1, 2, 0, 1, 1, 2, 2, 1, 2, 0, 1, 1, 0, 0,
       0, 1, 1, 0, 2, 2, 1, 2, 0, 2, 1, 1, 0, 2, 1, 2, 1, 0], dtype=int64)

In [75]:
X

array([[5.9, 3. , 5.1, 1.8],
       [5.4, 3. , 4.5, 1.5],
       [5. , 3.5, 1.3, 0.3],
       [5.6, 3. , 4.5, 1.5],
       [4.9, 2.5, 4.5, 1.7],
       [4.5, 2.3, 1.3, 0.3],
       [6.9, 3.1, 4.9, 1.5],
       [5.6, 2.7, 4.2, 1.3],
       [4.8, 3.4, 1.6, 0.2],
       [6.4, 3.2, 4.5, 1.5],
       [6.7, 3. , 5. , 1.7],
       [6. , 3.4, 4.5, 1.6],
       [5.2, 4.1, 1.5, 0.1],
       [7.2, 3.6, 6.1, 2.5],
       [5.2, 3.4, 1.4, 0.2],
       [5.9, 3.2, 4.8, 1.8],
       [6.7, 2.5, 5.8, 1.8],
       [6.4, 3.1, 5.5, 1.8],
       [5.1, 3.8, 1.6, 0.2],
       [4.9, 3.1, 1.5, 0.1],
       [5.8, 2.7, 3.9, 1.2],
       [6.9, 3.2, 5.7, 2.3],
       [6.1, 2.9, 4.7, 1.4],
       [6. , 2.2, 5. , 1.5],
       [7.2, 3. , 5.8, 1.6],
       [6. , 3. , 4.8, 1.8],
       [6.2, 2.9, 4.3, 1.3],
       [5.5, 2.4, 3.8, 1.1],
       [5.8, 2.7, 5.1, 1.9],
       [6.7, 3.1, 5.6, 2.4],
       [6.2, 2.8, 4.8, 1.8],
       [5.7, 2.6, 3.5, 1. ],
       [4.6, 3.6, 1. , 0.2],
       [6.9, 3.1, 5.4, 2.1],
       [6.4, 2

## Baseline model

In [27]:
# define baseline model
def baseline_model():

    model = models.Sequential()
    model.add(layers.Dense(8, activation='relu',
                          input_shape=(4,)))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model


In [28]:
estimator = KerasClassifier(build_fn=baseline_model, epochs=200, batch_size=5, verbose=0)

In [29]:
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

In [30]:
results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 96.67% (3.33%)


## Smaller Network

In [11]:
# smaller model
def create_smaller():

    model = models.Sequential()
    model.add(layers.Dense(4, activation='relu',
                          input_shape=(4,)))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    
    return model
estimator = KerasClassifier(build_fn=create_smaller, epochs=100, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 91.33% (13.01%)


## Larger Network

In [12]:
# larger model
def create_larger():
 
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu',
                          input_shape=(4,)))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
   

    
    return model

estimator = KerasClassifier(build_fn=create_larger, epochs=100, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 96.67% (3.33%)


## Overfit Model

In [13]:
def create_overfitting():
 
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu',
                          input_shape=(4,)))
    model.add(layers.Dense(8, activation='relu',
                          input_shape=(4,)))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
   

    
    return model

estimator = KerasClassifier(build_fn=create_overfitting, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 96.00% (5.33%)


## Optimum Model 

In [87]:
from keras.layers import Dropout

def create_optimum():
 
    model = models.Sequential()
    model.add(layers.Dense(16, activation='relu',
                          input_shape=(4,)))
    model.add(Dropout(0.2))
    model.add(layers.Dense(8, activation='relu',
                          input_shape=(4,)))
    model.add(Dropout(0.2))
    model.add(layers.Dense(3, activation='softmax'))
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
   

    
    return model

estimator = KerasClassifier(build_fn=create_optimum, epochs=200, batch_size=5, verbose=0)

kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

results = cross_val_score(estimator, X, dummy_y, cv=kfold)
print("Baseline: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))


Baseline: 98.00% (4.27%)


## Using Keras Functional API

In [15]:
from keras.layers import Input, Dense
from keras.models import Model

def functional_model():

 # This returns a tensor
 inputs = keras.Input(shape=(4,))

 # a layer instance is callable on a tensor, and returns a tensor
 x = Dense(8, activation='relu')(inputs)
 predictions = Dense(3, activation='softmax')(x)

 # This creates a model that includes
 # the Input layer and three Dense layers
 model = Model(inputs=inputs, outputs=predictions)
 model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])
 return model

#history = model.fit(X, encoded_Y, epochs=10, batch_size=30)  # starts training


In [16]:
import pandas 
import numpy 
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras import models
from keras import layers
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from sklearn import preprocessing
from sklearn.pipeline import Pipeline


# evaluate baseline model with standardized dataset
numpy.random.seed(seed)
estimators = []
estimators.append(('standardize', StandardScaler()))
history = estimators.append(('mlp', KerasClassifier(build_fn=functional_model, epochs=100, batch_size=5, verbose=0)))
pipeline = Pipeline(estimators)
kfold = StratifiedKFold(n_splits=10, shuffle=True, random_state=seed)
results = cross_val_score(pipeline, X, encoded_Y, cv=kfold)
print("Standardized: %.2f%% (%.2f%%)" % (results.mean()*100, results.std()*100))

Standardized: 95.33% (5.21%)


## Using Model Subclassing

In [85]:
import tensorflow as tf
import keras

#inputs = keras.Input(shape=(4,))

class MyModel(tf.keras.Model):

  def __init__(self):
    super(MyModel, self).__init__()
    self.dense1 = tf.keras.layers.Dense(8, activation=tf.nn.relu, input_shape=(4,))
    self.dense2 = tf.keras.layers.Dense(3, activation=tf.nn.softmax)

  def call(self, inputs):
    x = self.dense1(inputs)
    return self.dense2(x)

model = MyModel()

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

model.fit(X, encoded_Y, epochs=100, batch_size=5, verbose=0)



## k-fold Cross Validation without using scikit-learn

In [90]:
import numpy as np

k = 10
num_val_samples = len(X) // k
num_epochs = 200
all_scores = []
for i in range(k):
    print('processing fold #', i)
    # Prepare the encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    val_data = X[i * num_val_samples: (i + 1) * num_val_samples]
    val_targets = encoded_Y[i * num_val_samples: (i + 1) * num_val_samples]
    # Prepare the training data: data from all other partitions
    partial_train_data = np.concatenate(
        [X[:i * num_val_samples],
         X[(i + 1) * num_val_samples:]],
        axis=0)
    partial_train_targets = np.concatenate(
        [encoded_Y[:i * num_val_samples],
         encoded_Y[(i + 1) * num_val_samples:]],
        axis=0)

    # Build the Keras model (already compiled)
    model = create_optimum()
    
    model.compile(optimizer='adam',
    loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
    # Train the model (in silent mode, verbose=0)
    model.fit(partial_train_data, partial_train_targets, epochs=num_epochs, batch_size=5, verbose=0)
    #          epochs=num_epochs,validation_split=10, shuffle=True, batch_size=5, verbose=0)
    # Evaluate the model on the validation data
    val_err, val_acc = model.evaluate(val_data, val_targets, verbose=0)
    all_scores.append(val_acc)

processing fold # 0
processing fold # 1
processing fold # 2
processing fold # 3
processing fold # 4
processing fold # 5
processing fold # 6
processing fold # 7
processing fold # 8
processing fold # 9


In [91]:
###### all_scores
print("Standardized: %.2f%% (%.2f%%)" % (np.mean(all_scores)*100, np.std(all_scores)*100))


Standardized: 96.67% (4.47%)
